<a href="https://colab.research.google.com/github/MikeHankinson/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install PySpark** 
PySPark does not come native to Google Colab

---



In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [44.8 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:13 https://developer.download.nvi

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge_Deliverable2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

--2021-02-08 03:40:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.99MB/s    in 0.2s    

2021-02-08 03:40:59 (5.99 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



**Retrieve *vine_table.csv* from AWS S3**

---



In [148]:
from pyspark import SparkFiles
url = "https://hankmikebootcamp.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True)
df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

**Step 1:** Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.


---



In [149]:
total_votes20_df = df.filter("total_votes>=20")
total_votes20_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

**Step 2:** Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.


---

In [155]:
# Convert Pandas Df from Spark DF 
import pandas as pd
total_votes20_df = total_votes20_df.toPandas() 

# Convert helpful_votes and total_votes columns from string to float
total_votes20_df['helpful_votes'] = total_votes20_df['helpful_votes'].astype(float)
total_votes20_df['total_votes'] = total_votes20_df['total_votes'].astype(float)
total_votes20_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_ratio
0,R4PKAZRQJJX14,1,21.0,34.0,N,N,0.617647
1,R2CI0Y288CC7E2,1,21.0,35.0,N,Y,0.600000
2,R127WEQY2FM1T3,1,147.0,175.0,N,Y,0.840000
3,R3EZ0EPYLDA34S,1,14.0,31.0,N,Y,0.451613
4,R2FJ94555FZH32,2,55.0,60.0,N,N,0.916667


In [156]:
# Add "helpful/total" Column: helpful_votes divided by total_votes
total_votes20_df["helpful_ratio"] = (total_votes20_df["helpful_votes"] / total_votes20_df["total_votes"])


In [157]:
total_votes20_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_ratio
0,R4PKAZRQJJX14,1,21.0,34.0,N,N,0.617647
1,R2CI0Y288CC7E2,1,21.0,35.0,N,Y,0.600000
2,R127WEQY2FM1T3,1,147.0,175.0,N,Y,0.840000
3,R3EZ0EPYLDA34S,1,14.0,31.0,N,Y,0.451613
4,R2FJ94555FZH32,2,55.0,60.0,N,N,0.916667


In [158]:
# Convert Spark DF from Pandas Df 
from pyspark.sql import SparkSession
total_votes20_df = spark.createDataFrame(total_votes20_df)


In [145]:
total_votes20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| R4PKAZRQJJX14|          1|         21.0|       34.0|   N|                N|  0.6176470588235294|
|R2CI0Y288CC7E2|          1|         21.0|       35.0|   N|                Y|                 0.6|
|R127WEQY2FM1T3|          1|        147.0|      175.0|   N|                Y|                0.84|
|R3EZ0EPYLDA34S|          1|         14.0|       31.0|   N|                Y| 0.45161290322580644|
|R2FJ94555FZH32|          2|         55.0|       60.0|   N|                N|  0.9166666666666666|
|R1U3AR67RE273L|          1|         51.0|       65.0|   N|                Y|  0.7846153846153846|
|R3PZOXA5X1U8KW|          4|         31.0|       36.0|   N|                N|  0.8611111111111112|
| R6KTC1OP

In [159]:
# Filter helpful/total column >= 0.5
helpful50_df = total_votes20_df.filter("helpful_ratio>=0.5")
helpful50_df.show(20)




+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|         21.0|       34.0|   N|                N|0.6176470588235294|
|R2CI0Y288CC7E2|          1|         21.0|       35.0|   N|                Y|               0.6|
|R127WEQY2FM1T3|          1|        147.0|      175.0|   N|                Y|              0.84|
|R2FJ94555FZH32|          2|         55.0|       60.0|   N|                N|0.9166666666666666|
|R1U3AR67RE273L|          1|         51.0|       65.0|   N|                Y|0.7846153846153846|
|R3PZOXA5X1U8KW|          4|         31.0|       36.0|   N|                N|0.8611111111111112|
| R6KTC1OPIOIIG|          2|         19.0|       34.0|   N|                Y|0.5588235294117647|
|R36O341WWXXKNP|          5|  

**Step 3:** Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.


---

In [169]:
reviews_vine_df = helpful50_df.filter(helpful50_df["vine"] == "Y")
reviews_vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KKUSGFZWSUIY|          5|         56.0|       63.0|   Y|                N|0.8888888888888888|
|R10FO5UKKVZBK2|          3|         23.0|       23.0|   Y|                N|               1.0|
| RM4KSGEOR7MU1|          5|         19.0|       24.0|   Y|                N|0.7916666666666666|
| RG7VRMYLEXD23|          4|         22.0|       26.0|   Y|                N|0.8461538461538461|
|R11O4YSCPSNL6L|          3|         20.0|       26.0|   Y|                N|0.7692307692307693|
|R286MFBAJ8NPD6|          5|         46.0|       51.0|   Y|                N|0.9019607843137255|
|R1JRR530H4COA2|          5|         22.0|       28.0|   Y|                N|0.7857142857142857|
| RQ5WD90PUNBU9|          5|  

**Step 4:** Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

---

In [170]:
reviews_not_vine_df = helpful50_df.filter(helpful50_df["vine"] == "N")
reviews_not_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|         21.0|       34.0|   N|                N|0.6176470588235294|
|R2CI0Y288CC7E2|          1|         21.0|       35.0|   N|                Y|               0.6|
|R127WEQY2FM1T3|          1|        147.0|      175.0|   N|                Y|              0.84|
|R2FJ94555FZH32|          2|         55.0|       60.0|   N|                N|0.9166666666666666|
|R1U3AR67RE273L|          1|         51.0|       65.0|   N|                Y|0.7846153846153846|
|R3PZOXA5X1U8KW|          4|         31.0|       36.0|   N|                N|0.8611111111111112|
| R6KTC1OPIOIIG|          2|         19.0|       34.0|   N|                Y|0.5588235294117647|
|R36O341WWXXKNP|          5|  

**Step 5:** Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

---




In [178]:
# Total Reviews
vine_total_reviews = reviews_vine_df.count()
not_vine_total_reviews = reviews_not_vine_df.count()

In [179]:
#  Total 5 Star Reviews
vine_5_star = reviews_vine_df.filter(reviews_vine_df["star_rating"] == "5").count()
not_vine_5_star = reviews_not_vine_df.filter(reviews_not_vine_df["star_rating"] == "5").count()


In [193]:
# Percentage of 5 Star Reviews
vine_5_star_percent = vine_5_star/vine_total_reviews*100
non_vine_5_star_percent = (not_vine_5_star/not_vine_total_reviews*100)


In [194]:
# Create a Results DataFrame
vine_summary_df = pd.DataFrame([{"Vine Reviews":vine_total_reviews,
                                  "non-Vine Reviews":not_vine_total_reviews,
                                  "Vine 5-Star Reviews":vine_5_star,
                                  "non-Vine 5-Star Reviews":not_vine_5_star,
                                  "Vine 5-Star Reviews(%)":vine_5_star_percent,
                                  "non-Vine 5-Star Reviews(%)":non_vine_5_star_percent}])
                               


vine_summary_df.head()





,Vine Reviews,non-Vine Reviews,Vine 5-Star Reviews,non-Vine 5-Star Reviews,Vine 5-Star Reviews(%),non-Vine 5-Star Reviews(%)
0,94,40471,48,15663,51.06383,38.701786
